In [1]:
from thecodecrate_pipeline import (
    PipelineFactory,
    StageCollection,
)

# define a factory with a few stages
some_stages: StageCollection = (
    (lambda x: x + 1),
    (lambda x: x + 1),
    (lambda x: f"result is {x}"),
)

pipeline_factory = (PipelineFactory[int, str]()).with_stages(some_stages)

# create and process
pipeline = pipeline_factory.make()
result = await pipeline.process(10)

# check the result
assert result == "result is 12"
print(result)

result is 12


In [2]:
# adding another stage after the pipeline has been created
from typing import Any


def add_excitement(x: Any) -> str:
    return f"{x}!!!"


pipeline_factory.add_stage(add_excitement)

# the original pipeline is not affected
result2 = await pipeline.process(10)
assert result2 == result
print(f"original pipeline: {result}")

# now, let's create a new pipeline, with the excitement stage!!!
pipeline2 = pipeline_factory.make()
result3 = await pipeline2.process(10)

# the new pipeline has the excitement!!!
assert result3 == "result is 12!!!"
print(f"new pipeline: {result3}")

# notice the original pipeline is not affected
result4 = await pipeline.process(10)
assert result4 == result
print(f"original pipeline: {result}")

# what if we try to change the original pipeline, directly?
pipeline.pipe(add_excitement)  # <- adding the excitement stage to the original pipeline

# the original pipeline is still not affected, due to the immutability nature of pipelines
result5 = await pipeline.process(10)
assert result5 == result
print(f"original pipeline: {result}")

original pipeline: result is 12
new pipeline: result is 12!!!
original pipeline: result is 12
original pipeline: result is 12


In [3]:
from thecodecrate_pipeline import ChainedProcessor, StageCollection


# create factory with a processor
class MyProcessor(ChainedProcessor):
    pass


some_stages: StageCollection = (
    (lambda x: x + 1),
    (lambda x: x + 1),
    (lambda x: f"result is {x}"),
)

pipeline_factory = (
    (PipelineFactory[int, str]()).with_stages(some_stages).with_processor(MyProcessor())
)

# create and process
pipeline = pipeline_factory.make()
result = await pipeline.process(10)
print(result)

# check the processor
processor_instance = pipeline.get_processor_instance()
assert processor_instance.__class__ == MyProcessor
print(processor_instance)

result is 12
